## 2D Heat Conduction

The equation is:

$$\rho c_p \frac{\partial T}{\partial t} = \frac{\partial}{\partial x} \left( k_x \frac{\partial T}{\partial x} \right) + \frac{\partial}{\partial y} \left( k_y \frac{\partial T}{\partial y} \right)$$

Where $\rho$ is the density, $c_p$ is the heat capacity and $k$ is the thermal conductivity.

If the thermal conductivity $k$ is constant we can take it outside of the spatial derivative and the equation becomes:

$$\frac{\partial T}{\partial t} = \alpha \left( \frac{\partial ^2 T}{\partial x^2} + \frac{\partial ^2 T}{\partial y^2} \right)$$

Where $\alpha = \frac{k}{\rho c_p}$ is the thermal diffusivity.  The thermal diffusivity describes the ability of a material to conduct heat vs storing it.

It looks exactly like the diffusion equation.  That's why $\alpha$ is called the thermal diffusivity (duh).